In [19]:
from gym_go.envs import go_env
from gym_go import gogame
import torch 
from math import sqrt
import time



In [20]:
BOARD_SIZE=7
KOMI=0
L2_LOSS = 1
C_PUCT = 5
N_THRESHOLD = 10 
TEMPERATURE_MCTS = 2
N_TREE_SEARCH = 100
QU_RATIO = 0.5
EPSILON = 0.0001
MAX_TURNS = 200 # a bit plus then the total number of position to play

# 9*9 stats
# roll_time = 0.285

# 7*7 stats
# roll_time = 0.11

In [21]:
env = go_env.GoEnv(size=BOARD_SIZE, komi=KOMI)
state, reward, done, info = env.step((0,0))
print(state[:4], '\n----\n')
state, reward, done, info = env.step((1,0))
print(state[:4])


[[[1. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0.]]

 [[0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0.]]

 [[1. 1. 1. 1. 1. 1. 1.]
  [1. 1. 1. 1. 1. 1. 1.]
  [1. 1. 1. 1. 1. 1. 1.]
  [1. 1. 1. 1. 1. 1. 1.]
  [1. 1. 1. 1. 1. 1. 1.]
  [1. 1. 1. 1. 1. 1. 1.]
  [1. 1. 1. 1. 1. 1. 1.]]

 [[1. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0.]]] 
----

[[[1. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0.]]

 [[0. 0. 0. 0. 0. 0. 0.]
  [1. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0.]
  [0. 0.

In [22]:
class OutputFormating(torch.Tensor):
    @property
    def value(self):
        """Return the last element along dim=-1"""
        if self.ndim == 1:
            return self[-1]
        else:
            return self[..., -1]

    @property
    def result(self):
        """Return all elements except the last along dim=-1"""
        if self.ndim == 1:
            return self[:-1]
        else:
            return self[..., :-1]


In [23]:
class GoBot(torch.nn.Module):
  """
  See only the bord as playing as the black player 
  (so when it's white we just exchange the black and white color so it can see it as black)
  """
  def __init__(self):
    super(GoBot, self).__init__()

    # input channel = 1, output channels = 6, kernel size = 5
    # input image size = (32, 32), image output size = (28, 28)
    self.convol1 = torch.nn.Conv2d(4, 20, 7, padding=3, bias=True)
    
    self.convol2 = torch.nn.Conv2d(20, 40, 5, padding=2, bias=True)

    self.linear1 = torch.nn.Linear(40*(BOARD_SIZE**2), 150, bias=True)

    # input dim = 120, output dim = 84
    self.linear2 = torch.nn.Linear(150, 110, bias=True) 

    self.linear3 = torch.nn.Linear(110, BOARD_SIZE**2 + 2, bias=True) # 83 pour les 81 case possible le suivant pass et le dernier pour la valeur


  def forward(self, x):
    if not isinstance(x, torch.Tensor):
      x = torch.tensor(x, dtype=torch.float32)

    if x.ndim == 3: 
        x = x.unsqueeze(0)

    if x[:,2,0,0] == 1:
      x[:, [0,1], ...] = x[:, [1,0], ...]
    
    x = x[:,[0,1,3,4],...]


    x = torch.nn.ReLU() (self.convol1(x))

    x = torch.nn.ReLU() (self.convol2(x))

    # flatten the feature maps into a long vector
    x = x.view(x.shape[0], -1) 

    x = torch.nn.ReLU() (self.linear1(x))
    x = torch.nn.ReLU() (self.linear2(x))
    x = self.linear3(x)
    

    return OutputFormating(x)
  
  @property
  def value(self):
    return 

In [24]:
net = GoBot()
out = net.forward(state)
print(out.result[0])
print(out.result[0,10])



OutputFormating([-0.0743,  0.0326,  0.0644, -0.0639, -0.0690, -0.0412,  0.0746,
                 -0.0399,  0.1419,  0.0810, -0.0285, -0.0143, -0.0196, -0.0147,
                  0.0625,  0.0624, -0.0149,  0.0063, -0.0755,  0.0159,  0.0803,
                 -0.0018, -0.0229, -0.0707,  0.0241, -0.0831,  0.0141, -0.0475,
                  0.0286,  0.0260, -0.0856,  0.0180, -0.0300,  0.0534,  0.0886,
                  0.0096,  0.0918,  0.0553, -0.0478, -0.0473, -0.0197, -0.0840,
                  0.1155, -0.0036,  0.0403, -0.0831, -0.0013, -0.0379, -0.0367,
                 -0.0343], grad_fn=<AliasBackward0>)
OutputFormating(-0.0285, grad_fn=<AliasBackward0>)


In [25]:
def compute_loss(output, target, reward):
  value_loss = output.value - reward
  policy_loss = torch.dot(target, output.result)
  regulation = L2_LOSS
  return value_loss + policy_loss + regulation

In [26]:
def get_optimizer(net, lr, wd, momentum):
  optimizer = torch.optim.SGD(net.parameters(), lr=lr, weight_decay=wd, momentum=momentum)
  return optimizer

In [27]:
def test(net, data_loader, cost_function, device='cuda:0'):
  samples = 0.
  cumulative_loss = 0.
  cumulative_accuracy = 0.

  net.eval() # Strictly needed if network contains layers which has different behaviours between train and test
  with torch.no_grad():
    for batch_idx, (inputs, targets) in enumerate(data_loader):
      # Load data into GPU
      inputs = inputs.to(device)
      targets = targets.to(device)

      # Forward pass
      outputs = net(inputs)

      # Apply the loss
      loss = cost_function(outputs, targets)

      # Better print something
      samples+=inputs.shape[0]
      cumulative_loss += loss.item() # Note: the .item() is needed to extract scalars from tensors
      _, predicted = outputs.max(1)
      cumulative_accuracy += predicted.eq(targets).sum().item()

  return cumulative_loss/samples, cumulative_accuracy/samples*100


def train(net, data_loader, optimizer, device='cuda:0'):
  samples = 0.
  cumulative_loss = 0.


  net.train() # Strictly needed if network contains layers which has different behaviours between train and test
  for batch_idx, (state, targets, reward) in enumerate(data_loader):
    # Load data into GPU
    state = state.to(device)
    targets = targets.to(device)

    # Forward pass
    outputs = net(state)

    # Apply the loss
    loss = compute_loss(outputs, targets, reward)

    # Reset the optimizer

    # Backward pass
    loss.backward()

    # Update parameters
    optimizer.step()

    optimizer.zero_grad()

    # Better print something, no?
    samples+=state.shape[0]
    cumulative_loss += loss.item()
    # _, predicted = outputs[:82].max(1)   # for the action which has been remembered

  return cumulative_loss/samples

In [28]:
def action2d(action) :
    return action // BOARD_SIZE, action % BOARD_SIZE 

def roll_policy(state, net, prt = False):    
    """
    return :
        the score of the outcome of the self played game seen by the first player to play
    """
    player = state[2,0,0]
    game_ended = gogame.game_ended(state)

    game_turns = 0
    while not game_ended:
        if game_turns < MAX_TURNS :
            if torch.all(torch.tensor(state[3,:,:]) == 1):
                action = BOARD_SIZE**2
            else :
                result = net([state]).result[0]
                # print(result)
                sorted_indices_actions = sorted(range(len(result)), key=lambda i: result[i])
                invalid_moves = gogame.invalid_moves(state)
                for i in range(BOARD_SIZE**2+1) :
                    action = sorted_indices_actions[i]
                    if not invalid_moves[action] : 
                        break
        else :
            action = BOARD_SIZE**2
        
        if prt : print(action2d(action))
        if prt : print(state)
        state = gogame.next_state(state, action)
        game_ended = gogame.game_ended(state)
        game_turns += 1

    score = gogame.winning(state, KOMI)  # from absolut black perspective
    if player :
        score = - score
    return score        # from the perspective of the player who where supposed to play at the starting state of the roll_policy
    

In [29]:
state = gogame.init_state(BOARD_SIZE)
print(roll_policy(state, net))

1.0


In [30]:
net = GoBot()

In [31]:
class MCTS():
    root_depth = 0
    def __init__(self, state, action : int, p : torch.Tensor, depth = 0):
        self.next_nodes = []
        self.state = state
        self.action = action
        self.N = 0
        self.Wv = 0 
        self.Wr = 0 
        self.Q = 0
        self.p = p
        self.depth = depth

        self.first_search = True
        self.value = 0
        self.roll = 0

    def next_state(self):
        if self.action == None :
            return self.state
        else :
            return gogame.next_state(self.state, self.action)
    
    def action_2d(self):
        if self.action == None :
            return "None"
        else :
            return self.action // BOARD_SIZE, self.action % BOARD_SIZE


    def exctend_tree(self):
        # here the simulation of the accumulation of Wv and Wr will being just a leaf node
        self.Wv = self.N * self.value
        self.Wr = self.N * self.roll     
        next_state = self.next_state()
        result = net([next_state]).result[0]
        for i in range(BOARD_SIZE):
            for j in range(BOARD_SIZE):
                if next_state[3,i,j] == 1:
                    p = 0
                else : 
                    p = max(EPSILON,result[i*BOARD_SIZE + j])
                action = i*BOARD_SIZE + j
                # adding the mouve at possition (i,j)
                self.next_nodes.append(
                    MCTS(next_state, 
                        action, 
                        p,
                        self.depth + 1))
        # adding the pass mouve
        self.next_nodes.append(
            MCTS(next_state, 
                BOARD_SIZE**2, 
                EPSILON,
                self.depth + 1))


    def push_search(self, prt = False):
        if prt : print("self", self)
        if self.N >= N_THRESHOLD :
            if not self.next_nodes :    # extend node part
                print("extende : ", self.action_2d(), self.depth)
                self.exctend_tree()

            max_score = 0      # all scores can be <0 so can be improved here (maybe take the score of the default pass mouve)
            # max_score = self.next_nodes[-1].Q + c_puct * self.next_nodes[-1].p * sqrt(self.N)/(1+self.next_nodes[-1].N)
            node_to_search = self.next_nodes[-1]    # default next mouve is the pass mouve (used if no mouves has a positive score)
            if prt : print("___scores___")
            for next_node in self.next_nodes :
                score = next_node.Q + C_PUCT * next_node.p * sqrt(self.N)/(1+next_node.N)
                if prt : print(score)
                # print(score, next_node)
                if score > max_score :
                    max_score = score
                    node_to_search = next_node
            if prt : print("___scores_end___")
            value, output = node_to_search.push_search(prt)
            self.Wv += value  
            self.Wr += output 
            self.Q = ((1-QU_RATIO) * self.Wv + QU_RATIO * self.Wr)/self.N
            

        elif self.first_search :    # here self.Q is initialise and dont change the N_threshold first time but self.Wv and self.Wr should accumulate with is simulated in the extend node part
            if prt : print("___first_search___")
            self.first_search = False
            next_state = self.next_state()
            value = net(next_state).value
            if prt : print("__in_roll_out__")
            output = roll_policy(next_state, net, prt)
            if prt : print("___first_search_end___")
            
            self.value = value
            self.roll = output
            self.Q = (1-QU_RATIO) * self.value + QU_RATIO * self.roll     # don't change value until the expend happen

        else :
            value = self.value
            output = self.roll
    
        self.N += 1
        return - value, - output
    

    def best_next_node(self):
        max_visited_N = 0
        i = 0
        for next_node in self.next_nodes :
            i += 1
            visited_N = next_node.N
            if visited_N > max_visited_N :
                max_visited_N = visited_N
                best_node = next_node
        return best_node
    

    @staticmethod
    def tree_policy(node):
        actions = []
        visited_N = []
        for next_node in node.next_nodes :
            actions.append(next_node.action)
            visited_N.append(next_node.N)
        diviseur = sum(N**(1/TEMPERATURE_MCTS) for N in visited_N)
        sorted_index = sorted(range(len(actions)), key=lambda i : actions[i])
        MCTS_policy = [visited_N[i]/diviseur for i in sorted_index]
        return MCTS_policy
    
    def __str__(self):
        print("___ node info ___")
        print("action : {} | depth : {}".format(self.action_2d(), self.depth))
        print(self.N)
        # print(self.state[3])
        return ' '
        


In [32]:
state = gogame.init_state(BOARD_SIZE)
root = MCTS(state, None, 1)
for i in range(N_THRESHOLD + 3) :
    root.push_search()


extende :  None 0


In [33]:
def sample_one_self_play_MCTS(net):
    with torch.no_grad():
        data_set = []
        state = gogame.init_state(BOARD_SIZE)
        root = MCTS(state, None, 1)
        while not gogame.game_ended(state) : 
            print(root)
            tmp = time.time()
            
            for i in range(N_TREE_SEARCH) : 
                root.push_search()
            MCTS_policy = MCTS.tree_policy(root)
            data_set.append((state,MCTS_policy))
        
            next_root = root.best_next_node()
            next_state = gogame.next_state(state, next_root.action)
            root = next_root
            state = next_state
            print("\n", time.time() - tmp)
    return data_set
    

In [34]:
game_1 = sample_one_self_play_MCTS(net)

___ node info ___
action : None | depth : 0
0
 
extende :  None 0

 1.9630084037780762
___ node info ___
action : (3, 6) | depth : 1
7
 
extende :  (3, 6) 1

 2.4375603199005127
___ node info ___
action : (6, 1) | depth : 2
8
 
extende :  (6, 1) 2
extende :  (3, 1) 3
extende :  (5, 5) 3
extende :  (4, 0) 3
extende :  (5, 2) 3

 4.51160454750061
___ node info ___
action : (3, 1) | depth : 3
14
 
extende :  (6, 6) 4

 1.7330636978149414
___ node info ___
action : (6, 6) | depth : 4
11
 
extende :  (5, 3) 5

 1.9594132900238037
___ node info ___
action : (5, 3) | depth : 5
11
 
extende :  (1, 3) 6
extende :  (1, 2) 6

 1.9295179843902588
___ node info ___
action : (1, 2) | depth : 6
12
 
extende :  (1, 3) 7
extende :  (2, 4) 7

 1.4200503826141357
___ node info ___
action : (1, 3) | depth : 7
11
 
extende :  (1, 1) 8
extende :  (5, 0) 8
extende :  (2, 4) 8

 1.5529394149780273
___ node info ___
action : (1, 1) | depth : 8
12
 
extende :  (5, 0) 9

 1.3592798709869385
___ node info ___
act

In [35]:
print(game_1)

[(array([[[0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
    

In [36]:
def sample_self_play_MCTS_batchs(N, net, batch_size):
  for i in range(N):
    sample_one_self_play_MCTS(net)

#   training_data = torchvision.datasets.CIFAR10('./data', train=True, transform=transform, download=True)
  training_data = []

  # Initialize dataloaders
  train_loader = torch.utils.data.DataLoader(training_data, batch_size, shuffle=True)

  return train_loader

In [37]:
'''
Input arguments
  batch_size: Size of a mini-batch
  device: GPU where you want to train your network
  weight_decay: Weight decay co-efficient for regularization of weights
  momentum: Momentum for SGD optimizer
  epochs: Number of epochs for training the network
'''

def main(batch_size=128,
         device='cuda:0',
         learning_rate=0.01,
         weight_decay=0.000001,
         momentum=0.9,
         epochs=50,
         K_early_stopping = 5):

  train_loader = sample_data(batch_size, net)

  net = GoBot()

  optimizer = get_optimizer(net, learning_rate, weight_decay, momentum)

  print('Before training:')
  train_loss, train_accuracy = test(net, train_loader, device)


  print('\t Training loss {:.5f}, Training accuracy {:.2f}'.format(train_loss, train_accuracy))
  print('-----------------------------------------------------')

  for e in range(epochs):
    train_loss, train_accuracy = train(net, train_loader, optimizer, device)
    
    print('Epoch: {:d}'.format(e+1))
    print('\t Training loss {:.5f}, Training accuracy {:.2f}'.format(train_loss, train_accuracy))
    print('-----------------------------------------------------')

  print('After training:')
  train_loss, train_accuracy = test(net, train_loader, device)

  print('\t Training loss {:.5f}, Training accuracy {:.2f}'.format(train_loss, train_accuracy))
  print('-----------------------------------------------------')
  return net